In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from search_libraries import *

@interact(search="")
def search(search):
    """
    Widget for library search.
    
    Search Yarra, Melbourne and Moreland library catalogues with the search term entered in the search box.
    
    Args:
        search (str): The search term
    Returns:
        list: The list of zipped (as in zip()) results for catalogue searches.
    """
    titles = []
    if len(search) > 3:
        zipped = zip(search_yarra(search), search_melbourne(search), search_moreland(search))
        for yarra, melb, moreland in zipped:
            titles.append(yarra)
            titles.append(melb)
            titles.append(moreland)
    return titles

A Jupyter Widget